Locality Sensitive Hashing (LSH)의 실습을 위해 50차원 크기의 인공데이터 100만개를 만듦

In [4]:
mm_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/models/2016-10-20_noun_x.mm.mtx'
corpus_fname = '/home/paulkim/workspace/python/Korean_NLP/data/corpus_10days/news/2016-10-20_article_all_normed.txt'

mmread(fname)의 return type은 coo_matrix이며, 이는 slice가 안됨. 그렇기 때문에 csr_matrix로 바꿔야함

x.shape와 len(docs)의 길이가 맞는지도 확인

In [5]:
from scipy.io import mmread
x = mmread(mm_fname).tocsr()

with open(corpus_fname, encoding='utf-8') as f:
    docs = [doc.strip() for doc in f]
    
print(x.shape, len(docs))

(30091, 2611) 30091


임의의 문서 하나를 선택함. 미르재단과 최순실과의 관계에 대한 뉴스

이 뉴스의 term frequency vector는 x[2317, :]으로 선택할 수 있음

In [6]:
docs[2317]

'서울 연합뉴스 비선 실세 논란의 중심에 선 최순실씨를 둘러싼 각종 의혹이 난무하고 있다 전경련이 800억 원 가까운 기금을 내 만든 문화 스포츠 재단인 미르와 스포츠 설립 및 인사 관련 논란으로 출발한 최순실씨의 재단 사유화 의혹은 정경유착 논쟁으로 번지면서 최대 재계 단체인 전국경제인연합회의 해체 압력으로 이어졌고 최씨 딸의 이화여대 체육특기자 입학과 성적 등 학사관리를 둘러싼 특혜 의혹은 최경희 총장의 사퇴를 부르는 등 파문이 확산되고 있다 최씨가 사실상의 소유주라는 의심을 받는 더블루케이와 최씨 모녀의 개인회사인 비텍스포츠는 스포츠재단을 통해 들어온 돈을 사적으로 쓰려 했다는 보도까지 나왔다 여기에 최씨 모녀의 부적절한 언동이 를 타고 퍼지면서 여론이 악화하고 있다  청와대는 미르 스포츠재단 기금 모금과 관련한 청와대 연루설에 대해서는 이미 사실무근 이라고 부인하면서 야권의 부당한 정치 공세라고 반박한 바 있다 전경련도 미르와 스포츠는 기업들이 자발적으로 설립한 재단으로 권력의 개입은 없었다고 밝혔다 하지만 의혹이 수그러들기는커녕 갈수록 가지를 뻗고 있어 이 문제를 그냥 넘길 수 없는 상황이 됐다 여권 내부에서조차 비주류를 중심으로 권력형 비리 파문으로 확산하는 것을 차단하기 위해서라도 검찰 수사 등을 통해 신속히 진상을 규명해야 한다는 목소리가 커지고 있다 일각에서는 미르 스포츠재단 문제가 블랙홀처럼 모든 현안을 빨아들이는 판에 집권 여당이 진상 규명을 막는 듯한 모습을 보이는 것은 향후 국정 운영에 부담이 될 것이라고 우려한다 야권에서는 지금까지 불거진 최씨와 관련한 각종 의혹을 최순실 게이트 로 규정하고 국정조사와 특검까지 거론하고 있다  박근혜 대통령은 이와 관련 20일 수석비서관회의에서 어느 누구라도 재단과 관련해 자금 유용 등 불법 행위를 저질렀다면 엄정하게 처벌받을 것 이라고 밝혔다 한 달 전 미르 스포츠재단 의혹이 불거진 이후 박 대통령이 이처럼 구체적으로 언급한 건 처음이다 박 대통령은 문화 체육 분야를 집중 지원하고 우리 문화를 알리며

LSHForest를 이용하여 Cosine distance 기준으로 유사한 문서를 찾아봄.

In [7]:
%%time

from sklearn.neighbors import LSHForest
lsh = LSHForest()
lsh.fit(x)

/home/paulkim/anaconda3/envs/python3_0901/lib/python3.6/site-packages/sklearn/neighbors/approximate.py:220: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


CPU times: user 374 ms, sys: 0 ns, total: 374 ms
Wall time: 372 ms


2317번 문서와 유사한 10개의 문서를 선택함. 가장 비슷한 문서는 2317번이어야 함

In [8]:
dist, idxs = lsh.kneighbors(x[2317, :], n_neighbors=10)

2317번을 포함한 10개의 문서에 대하여 각 문서 별 200글자까지만 snippest를 출력하여 그 내용을 확인함. 이와 같이 비슷한 문서를 찾는 문제에서는 LSHForest를 쓰면 편리함

또한, 문서는 어느 정도 길이가 긴 문선인 것이 좋음. 짧은 문서는 too much sparse하여 인덱싱이 잘 되지 않을 수 있음

In [9]:
for similar_doc in idxs[0]:
    print('%s ...\n'%docs[similar_doc][:200])

서울 연합뉴스 비선 실세 논란의 중심에 선 최순실씨를 둘러싼 각종 의혹이 난무하고 있다 전경련이 800억 원 가까운 기금을 내 만든 문화 스포츠 재단인 미르와 스포츠 설립 및 인사 관련 논란으로 출발한 최순실씨의 재단 사유화 의혹은 정경유착 논쟁으로 번지면서 최대 재계 단체인 전국경제인연합회의 해체 압력으로 이어졌고 최씨 딸의 이화여대 체육특기자 입학과 성 ...

한겨레 검찰 수사로 정권 불똥 막기  의미있는 사업에 인신공격  강제모금 초고속 설립 모르쇠  퇴임 대비한 재단 의혹 부인  박근혜 대통령이 20일 오후 청와대에서 열린 수석비서관회의에서 참석자들에게 자리를 안내하고 있다 왼쪽으로 안종범 정책조정수석과 그 뒤로 우병우 민정수석이 보인다 청와대사진기자단  박근혜 대통령이 20일 비선 실세 로 알려진 최순실 6 ...

미르 스포츠재단 의혹 관련  검찰에 철저한 수사 지시  퇴임후 대비 사실 아냐 일축  최순실 논란 침묵 깨고 첫 언급  기업들 뜻 모아 만든 재단  강제모금설 부인 수사 영향  박근혜 대통령은 20일 대기업들이 낸 미르 스포츠재단 자금을 최순실 60 최서원으로 개명 씨가 빼돌려 쓰고 재단을 사유화했다는 의혹에 대해 어느 누구라도 재단과 관련해 자금 유용 등 ...

수석회의 모두 발언 절반 할애 기업들 자발적으로 자금 모금 검찰에 수사 가이드라인 제시 최씨 재단설립 개입 여부 언급 안해 야 공세에 인신공격성 논란 규정 사실상 최순실 감싸기 지적 일어  박근혜 대통령이 20일 미르 스포츠 재단 설립 과정과 모금 경위에 대해 설명한 것은 자신의 최측근인 최순실씨가 두 재단 의혹의 중심에 있다는 파장이 확산되는 것을 차단하 ...

수석비서관회의에서 인신공격성 논란 기업의 순수한 참여의지로 이뤄진 문화융성 사업 이화여대 특혜는 언급 없어 미디어오늘 이재진 기자 박근혜 대통령이 최순실씨 의혹을 도가 지나친 인식 공격성 논란으로 규정했다  박 대통령은 20일 청와대에서 주재한 수석비서관회의에서 의미있는 사업에 대해 의혹이 확산되고 도를 지나치게 